Выходной сигнал коррелятора спектрометра DLS-82E определяется выражением:
\begin{equation}
    \label{eq:eq1}
    S\left(\tau,C_A,F_0, t_1\right) = C_A K_{BS} K_{LS} \phi\left(\tau,F_0,t_1\right),
\end{equation}
где  
$C_A$ - амплитуда емкостного релаксационного сигнала,  
$K_{BS}$ - масштабный коэффициент, зависящий от чувствительности емкостного моста,  
$K_{LS}$ - масштабный коэффициент селектора,  
$\tau$ - постоянная времени релаксации гулбокого уровня,  
$F_0$ - частота сканирования импульсов заполнения,  
$t_1$ - длительность импульса заполнения,  
$\phi\left(\tau,F_0,t_1\right)$ - функция определяемая выражением:  
\begin{equation}
    \label{eq:eq2}
    \phi\left(\tau,F_0,t_1\right) = 
    M \tau F_0 e^{-\frac{0.05}{\tau F_0}}
    \left(1-e^{\frac{t_1 F_0-0.45}{\tau F_0}}
    -e^{-\frac{0.5}{\tau F_0}}+
    e^{\frac{t_1 F_0-0.95}{\tau F_0}}\right),
\end{equation}
где $M$ -- масштабный множитель.

Таким образом,
$$  M = \frac{1}{\max{\left[
    \tau F_0 e^{-\frac{0.05}{\tau F_0}}
    \left(1-e^{\frac{t_1 F_0-0.45}{\tau F_0}}
    -e^{-\frac{0.5}{\tau F_0}}+
    e^{\frac{t_1 F_0-0.95}{\tau F_0}}\right)
    \right]}}
$$

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

%matplotlib inline

$$ \tau = 10^{power_\tau} $$
$$ a = \tau F_0 $$
$$ b = t_1 F_0 $$
$$ exp0 = \exp{\left(\frac{-0.05}{a}\right)} $$
$$ exp1 = \exp{\left(\frac{b-0.45}{a}\right)} $$
$$ exp2 = \exp{\left(\frac{-0.5}{a}\right)} $$
$$ exp3 = \exp{\left(\frac{b-0.95}{a}\right)} $$
$$ \phi = a \cdot exp0 \cdot \left(1.0 - exp1 - exp2 + exp3\right) $$

In [2]:
def get_phi(frequency, time_constant_power, M, filling_pulse):
    time_constant = tf.pow(10.0, time_constant_power)

    a = time_constant * frequency
    b = filling_pulse * frequency

    exp0 = tf.exp(-0.05 / (a))
    exp1 = tf.exp((b - 0.45) / (a))
    exp2 = tf.exp(-0.5 / (a))
    exp3 = tf.exp((b - 0.95) / (a))

    phi = M * a * exp0 * (1.0 - exp1 - exp2 + exp3)

    return phi


def get_dlts(amplitude, M, frequency, time_constant_power, filling_pulse):
    dlts = amplitude * get_phi(frequency, 
                               time_constant_power, 
                               M, 
                               filling_pulse)
    
    return dlts

In [3]:
amplitude = tf.Variable(3.0, dtype='float64', trainable=True)
M = tf.Variable(5.861, dtype='float64', trainable=True)
time_constant_power = tf.Variable(0.001, dtype='float64', trainable=True)

# start_f = 1.0
# stop_f = 2500.0
# num_ex = 100

# frequency_powers = tf.linspace(np.log10(start_f), np.log10(stop_f), num_ex)
# frequency = tf.pow(10.0, frequency_powers)

frequency = tf.Variable(1000.0, dtype='float64', trainable=False)

filling_pulse = tf.Variable(20*10^(-6), dtype='float64', trainable=False)

In [4]:
# frequency_powers.dtype

In [5]:
# with tf.GradientTape(persistent=True) as tape:
#     S = get_dlts(amplitude, 
#                  M, 
#                  frequency, 
#                  time_constant_power, 
#                  filling_pulse)
    
#     phi = get_phi(frequency, 
#                   time_constant_power, 
#                   M,
#                   filling_pulse)

In [6]:
with tf.GradientTape() as S_tape:
    S = get_dlts(amplitude, 
                 M, 
                 frequency, 
                 time_constant_power, 
                 filling_pulse)

with tf.GradientTape() as phi_tape:
    phi = get_phi(frequency, 
                  time_constant_power, 
                  M,
                  filling_pulse)

In [7]:
# dSdtau_power = tape.gradient(S, [time_constant_power])
# dSdA = tape.gradient(S, [amplitude])
# dphidM = tape.gradient(phi, [M])

In [8]:
jacobians = S_tape.gradient(S, [time_constant_power, amplitude])
dphidM = phi_tape.gradient(phi, [M])

In [9]:
jacobians

[<tf.Tensor: shape=(), dtype=float64, numpy=0.00605675279826667>,
 <tf.Tensor: shape=(), dtype=float64, numpy=2.9296230336190394>]

In [10]:
dphidM

[<tf.Tensor: shape=(), dtype=float64, numpy=0.4998503725676573>]